# Convolutional Neural Networks

**Time**
- Teaching: 2.5 hours
- Challenges: 30 minutes

**Questions**
- "Understand "


**Learning Objectives**
- ""

* * * * *

## Import packages

For this notebook, instead of importing specific functions, we will import modules that contain functions.

**Old way:**

`from keras.layers import Dense`

`model.add(Dense(...))`

**New way:**

`from keras import layers`

`model.add(layers.Dense(...))`

But why change it up? I had trouble myself in the past understanding the way modules work, so code would break due simply to the way imports were done. Let's avoid that by getting comfortable with python modules!

In [1]:
from keras import layers
from keras import models

## Hardware for Deep-Learning

In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8786908788395665116
]


2022-02-09 15:05:01.076002: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-09 15:05:01.083220: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-09 15:05:01.083273: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-09 15:05:01.083304: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-seanperez): /proc/driver/nvidia/version does not exist
